In [2]:
!pip install pymysql

In [1]:
import pandas as pd
import pymysql
import numpy as np
from flask import Flask, redirect, url_for, request
from sklearn.metrics import mean_squared_error

user_list = []
tattoo_list = []

def select(var):
    conn = pymysql.connect(host='220.93.165.169', user='tmakxmaleldj', password='wlqdprkrhtlvek1!', db='tattoo', charset='utf8')
    try:
        with conn.cursor() as curs:
            sql = "select * from rating"
            curs.execute(sql)
            rs = curs.fetchall()
            for row in rs:
                user_list.append(row)
                print(row)
    finally:
        conn.close()
"""def select2(var):
    conn = pymysql.connect(host='220.93.165.169', user='tmakxmaleldj', password='wlqdprkrhtlvek1!', db='tattoo', charset='utf8')
    try:
        with conn.cursor() as curs:
            sql = "select * from tattoo"
            curs.execute(sql)
            rs = curs.fetchall()
            for row in rs:
                tattoo_list.append(row)
                print(row)
    finally:
        conn.close()"""

def get_rmse(R, P, Q, non_zeros):
    error = 0
    full_pred_matrix = np.dot(P, Q.T)
    
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))

    break_count = 0
       
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
   
    for step in range(steps):
        for i, j, r in non_zeros:
            eij = r - np.dot(P[i, :], Q[j, :].T)
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
       
        rmse = get_rmse(R, P, Q, non_zeros)
        
    return P, Q

def get_unseen_tattoos(ratings_matrix, userId):
    user_rating = ratings_matrix.loc[userId,:]
    already_seen = user_rating[ user_rating > 0].index.tolist()
    
    tattoos_list = ratings_matrix.columns.tolist()
    
    unseen_list = [ tattoo for tattoo in tattoos_list if tattoo not in already_seen]
    
    return unseen_list

def recomm_tattoo_by_userid(pred_df, userId, unseen_list, top_n=10):
    recomm_tattoos = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]

    return recomm_tattoos

In [ ]:
app = Flask(__name__)
@app.route('/', methods = ['POST', 'GET'])

def recommend():    
    if request.method == 'POST':
        name = request.form['id']
    else:
        name = request.args.get('id')

    select(())

    ratings = pd.DataFrame(user_list)
    ratings.columns = ['member_member_id', 'tattoo_tattoo_id', 'rating']

    tattoos = pd.read_csv('./tattoo.csv', encoding = 'EUC-KR')
    tattoos.columns = ['tattoo_tattoo_id', 'category_id', 'category_name']
    tattoos = tattoos[['tattoo_tattoo_id', 'category_id']]

    ratings_matrix = ratings.pivot_table('rating', index='member_member_id', columns='tattoo_tattoo_id')
    rating_tattoos = pd.merge(ratings, tattoos, on='tattoo_tattoo_id')
    ratings_matrix = rating_tattoos.pivot_table('rating', index='member_member_id', columns='tattoo_tattoo_id')
    P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=210, learning_rate=0.01, r_lambda = 0.01)
    pred_matrix = np.dot(P, Q.T)
    ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

    unseen_list = get_unseen_tattoos(ratings_matrix, name)
    recomm_tattoos = recomm_tattoo_by_userid(ratings_pred_matrix, name , unseen_list, top_n=10)
    recomm_tattoos = pd.DataFrame(data=recomm_tattoos.values,index=recomm_tattoos.index,columns=['pred_score'])

    conn = pymysql.connect(host='220.93.165.169', user='tmakxmaleldj', password='wlqdprkrhtlvek1!', db='tattoo', charset='utf8')

    str = [name]

    a = recomm_tattoos.index[:5]

    tattoo_1 = {'member_member_id' : str, 'tattoo_tattoo_id' : a.values[0]}
    survey_result_1 = pd.DataFrame((tattoo_1))
    rows_1 = [tuple(x) for x in survey_result_1.to_records(index = False)]

    tattoo_2 = {'member_member_id' : str, 'tattoo_tattoo_id' : a.values[1]}
    survey_result_2 = pd.DataFrame((tattoo_2))
    rows_2 = [tuple(x) for x in survey_result_2.to_records(index = False)]

    tattoo_3 = {'member_member_id' : str, 'tattoo_tattoo_id' : a.values[2]}
    survey_result_3 = pd.DataFrame((tattoo_3))
    rows_3 = [tuple(x) for x in survey_result_3.to_records(index = False)]

    tattoo_4 = {'member_member_id' : str, 'tattoo_tattoo_id' : a.values[3]}
    survey_result_4 = pd.DataFrame((tattoo_4))
    rows_4 = [tuple(x) for x in survey_result_4.to_records(index = False)]

    tattoo_5 = {'member_member_id' : str, 'tattoo_tattoo_id' : a.values[4]}
    survey_result_5 = pd.DataFrame((tattoo_5))
    rows_5 = [tuple(x) for x in survey_result_5.to_records(index = False)]

    cursor = conn.cursor()

    print(rows_1)
    sql = "insert into survey_result_1 values(%s, %s)"
    cursor.executemany(sql, rows_1)  
    print(cursor.rowcount, "record inserted.\n")

    print(rows_2)
    sql = "insert into survey_result_2 values(%s, %s)"
    cursor.executemany(sql, rows_2)  
    print(cursor.rowcount, "record inserted.\n")

    print(rows_3)
    sql = "insert into survey_result_3 values(%s, %s)"    
    cursor.executemany(sql, rows_3)  
    print(cursor.rowcount, "record inserted.\n")

    print(rows_4)
    sql = "insert into survey_result_4 values(%s, %s)" 
    cursor.executemany(sql, rows_4)   
    print(cursor.rowcount, "record inserted.\n")

    print(rows_5)
    sql = "insert into survey_result_5 values(%s, %s)"    
    cursor.executemany(sql, rows_5)   
    print(cursor.rowcount, "record inserted.\n")

    cursor.close()             
    conn.commit()
    conn.close()
    

if __name__ == "__main__":
    app.run(host="localhost", port="9000")   

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9000/ (Press CTRL+C to quit)


('member01', 'tattoo_785', 2)
('member01', 'tattoo_649', 4)
('member01', 'tattoo_475', 2)
('member01', 'tattoo_262', 2)
('member01', 'tattoo_679', 3)
('member01', 'tattoo_777', 2)
('member01', 'tattoo_805', 2)
('member01', 'tattoo_469', 4)
('member01', 'tattoo_458', 3)
('member01', 'tattoo_609', 2)
('member01', 'tattoo_453', 2)
('member01', 'tattoo_70', 3)
('member01', 'tattoo_792', 4)
('member01', 'tattoo_387', 5)
('member01', 'tattoo_502', 3)
('member01', 'tattoo_49', 2)
('member01', 'tattoo_558', 2)
('member01', 'tattoo_406', 3)
('member01', 'tattoo_337', 4)
('member01', 'tattoo_111', 3)
('member02', 'tattoo_489', 3)
('member02', 'tattoo_101', 2)
('member02', 'tattoo_396', 4)
('member02', 'tattoo_605', 3)
('member02', 'tattoo_396', 2)
('member02', 'tattoo_424', 3)
('member02', 'tattoo_99', 5)
('member02', 'tattoo_681', 5)
('member02', 'tattoo_272', 2)
('member02', 'tattoo_425', 5)
('member02', 'tattoo_22', 4)
('member02', 'tattoo_652', 3)
('member02', 'tattoo_755', 3)
('member02', '

[2020-11-25 15:44:27,763] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "C:\Users\SM120\anaconda3\envs\deep\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\SM120\anaconda3\envs\deep\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\SM120\anaconda3\envs\deep\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\SM120\anaconda3\envs\deep\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\SM120\anaconda3\envs\deep\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\SM120\anaconda3\envs\deep\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-2-b0cb4

('basisi1989', 'tattoo_579', 3)
('basisi1989', 'tattoo_202', 3)
('basisi1989', 'tattoo_134', 3)
('basisi1989', 'tattoo_434', 3)
('basisi1989', 'tattoo_815', 3)
('basisi1989', 'tattoo_471', 3)
('basisi1989', 'tattoo_725', 3)
('basisi1989', 'tattoo_184', 3)
('basisi1989', 'tattoo_748', 3)
('basisi1989', 'tattoo_417', 3)
('basisi1989', 'tattoo_132', 3)
('basisi1989', 'tattoo_475', 3)
('basisi1989', 'tattoo_587', 3)
('basisi1989', 'tattoo_79', 3)
('basisi1989', 'tattoo_556', 3)
('basisi1989', 'tattoo_612', 3)
('basisi1989', 'tattoo_794', 3)
('basisi1989', 'tattoo_202', 3)
('basisi1989', 'tattoo_556', 3)
('basisi1989', 'tattoo_612', 3)
('basisi1989', 'tattoo_794', 3)
('basisi1989', 'tattoo_202', 3)
('basisi1989', 'tattoo_300', 3)
('basisi1989', 'tattoo_453', 3)
('basisi1989', 'tattoo_769', 3)
('basisi1989', 'tattoo_294', 3)
('basisi1989', 'tattoo_268', 3)
('basisi1989', 'tattoo_1', 3)
('basisi1989', 'tattoo_584', 3)
('basisi1989', 'tattoo_614', 3)
('basisi1989', 'tattoo_425', 3)
('basisi198

[2020-11-25 15:44:45,081] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "C:\Users\SM120\anaconda3\envs\deep\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\SM120\anaconda3\envs\deep\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\SM120\anaconda3\envs\deep\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\SM120\anaconda3\envs\deep\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\SM120\anaconda3\envs\deep\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\SM120\anaconda3\envs\deep\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-2-b0cb4

('basisi1989', 'tattoo_579', 3)
('basisi1989', 'tattoo_202', 3)
('basisi1989', 'tattoo_134', 3)
('basisi1989', 'tattoo_434', 3)
('basisi1989', 'tattoo_815', 3)
('basisi1989', 'tattoo_471', 3)
('basisi1989', 'tattoo_725', 3)
('basisi1989', 'tattoo_184', 3)
('basisi1989', 'tattoo_748', 3)
('basisi1989', 'tattoo_417', 3)
('basisi1989', 'tattoo_132', 3)
('basisi1989', 'tattoo_475', 3)
('basisi1989', 'tattoo_587', 3)
('basisi1989', 'tattoo_79', 3)
('basisi1989', 'tattoo_556', 3)
('basisi1989', 'tattoo_612', 3)
('basisi1989', 'tattoo_794', 3)
('basisi1989', 'tattoo_202', 3)
('basisi1989', 'tattoo_556', 3)
('basisi1989', 'tattoo_612', 3)
('basisi1989', 'tattoo_794', 3)
('basisi1989', 'tattoo_202', 3)
('basisi1989', 'tattoo_300', 3)
('basisi1989', 'tattoo_453', 3)
('basisi1989', 'tattoo_769', 3)
('basisi1989', 'tattoo_294', 3)
('basisi1989', 'tattoo_268', 3)
('basisi1989', 'tattoo_1', 3)
('basisi1989', 'tattoo_584', 3)
('basisi1989', 'tattoo_614', 3)
('basisi1989', 'tattoo_425', 3)
('basisi198